<a href="https://colab.research.google.com/github/rayyanc/marketing-pages/blob/main/SUNBURST_DGA_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
SUNBURST DGA Decoder Walkthrough
Based on Erik Hjelmvik's decoder from Netresec and SolarWinds incident analysis
This notebook demonstrates how the SUNBURST malware encoded victim information in DNS queries
"""

import struct
import base64
from datetime import datetime, timedelta
from collections import defaultdict
from enum import IntFlag
import sys

In [ ]:
"""
Cell 1: Introduction to SUNBURST Domain Generation Algorithm (DGA)
The SUNBURST backdoor communicated with C2 servers using encoded DNS queries.
Each DNS query contained encoded information about the victim organization.
"""

print("=" * 80)
print("SUNBURST Domain Generation Algorithm (DGA) Decoder")
print("Educational Walkthrough for Security Analysis")
print("=" * 80)
print()

# Load the domain list from file
def load_domains_from_file(filename='???'): #FILL IN THE BLANK
    """Load domains from the uniq-hostnames.txt file"""
    try:
        with open(filename, 'r') as f:
            domains = [line.strip() for line in f if line.strip()]
        print(f"Loaded {len(domains)} domains from {filename}")
        return domains
    except FileNotFoundError:
        print(f"Warning: {filename} not found. Using sample domains instead.")
        return [
            "5qbtj04rcbp3tiq8bo6t.appsync-api.us-west-2.avsvmcloud.com",
            "6a57jk2ba1d9keg15cbg.appsync-api.eu-west-1.avsvmcloud.com",
            "7sbvaemscs0mc925tb99.appsync-api.us-west-2.avsvmcloud.com",
        ]

domains = load_domains_from_file()
print(f"\nFirst 5 domains for analysis:")
for domain in domains[???]: #FILL IN THE BLANK
    print(f"  {domain}")
print()

SyntaxError: invalid syntax (ipython-input-2691107022.py, line 31)

In [ ]:
"""
Cell 2: Understanding the Domain Structure
Each SUNBURST domain has a specific structure:
- First 15 characters: Encrypted GUID (unique identifier)
- Character 16: Segment number (for multi-part messages)
- Remaining characters: Encoded victim domain or security product info
"""

# Define the encoding alphabets used by SUNBURST
SUBSTITUTION_ALPHABET = "rq3gsalt6u1iyfzop572d49bnx8cvmkewhj"
STANDARD_BASE32_ALPHABET = "???" #FILL IN THE BLANK
BASE32_ALPHABET = "ph2eifo3n5utg1j8d94qrvbmk0sal76c"
SPECIAL_CHARS = "0_-."
MAX_SEGMENTS = ??? #FILL IN THE BLANK

print("SUNBURST Encoding Schemes:")
print("-" * 60)
print(f"Substitution Cipher Alphabet: {SUBSTITUTION_ALPHABET}")
print(f"Base32 Alphabet:              {BASE32_ALPHABET}")
print(f"Special Characters:           {SPECIAL_CHARS}")
print(f"Max Domain Segments:          {MAX_SEGMENTS}")
print()

SyntaxError: invalid syntax (ipython-input-4245931907.py, line 14)

In [ ]:
"""
Cell 3: Anti-Virus Product Detection
SUNBURST could report which security products were running on the victim system.
This information was encoded in special DNS queries.
"""

class AntiVirusStatus(IntFlag):
    """Enumeration of security products monitored by SUNBURST"""
    WindowsDefender_RUNNING = 0x0001
    WindowsDefender_STOPPED = 0x0002
    WindowsDefenderATP_RUNNING = 0x0004
    WindowsDefenderATP_STOPPED = 0x0008
    MicrosoftAzureATP_RUNNING = 0x0010
    MicrosoftAzureATP_STOPPED = 0x0020
    CarbonBlack_RUNNING = 0x0040
    CarbonBlack_STOPPED = 0x0080
    CrowdStrike_RUNNING = 0x0100
    CrowdStrike_STOPPED = 0x0200
    FireEye_RUNNING = 0x0400
    FireEye_STOPPED = 0x0800
    ESET_RUNNING = 0x1000
    ESET_STOPPED = 0x2000
    FSecure_RUNNING = 0x4000
    FSecure_STOPPED = 0x8000

class SzType:
    """Types of status messages"""
    Ping = 1
    AVProducts = 2

print("Security Products Tracked by SUNBURST:")
print("-" * 60)
for status in AntiVirusStatus:
    product, state = status.name.split('???') #FILL IN THE BLANK
    print(f"  {product:???} - {state}")  #FILL IN THE BLANK
print()

Security Products Tracked by SUNBURST:
------------------------------------------------------------


ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
"""
Cell 4: Base32 Decoder Implementation
The Base32 decoder uses a custom alphabet to decode binary data.
Each character represents 5 bits of information.
"""

def base32_decode_binary(encoded_string, skip_bits=0):
    """
    Decode Base32 string using SUNBURST's custom alphabet.

    Args:
        encoded_string: The Base32 encoded string
        skip_bits: Number of bits to skip at the beginning (for fragment reassembly)

    Returns:
        Decoded bytes
    """
    # Build reverse lookup dictionary (character -> 5-bit value)
    reverse_dict = {c: i for i, c in enumerate(BASE32_ALPHABET)}

    # Keep only valid characters
    encoded_string = ''.join(c for c in encoded_string if c in reverse_dict)

    # Early return if input is empty
    if not encoded_string:
        return b''

    buffer = 0      # Accumulates bits
    bit_count = 0   # Number of bits currently in the buffer
    result = []     # Output bytes

    for c in encoded_string:
        value = reverse_dict[c]  # Each Base32 character represents ??? bits #FILL IN THE BLANK

        # Align new 5-bit value into buffer considering skip_bits
        if skip_bits > bit_count:
            # Right shift: align value if buffer has fewer bits than skip_bits
            buffer |= (value >> (skip_bits - bit_count))
        else:
            # Left shift: append value to existing bits in buffer
            buffer |= (value << (bit_count - skip_bits))

        bit_count += ???  # How many bits did we just add? #FILL IN THE BLANK

        # Extract full bytes whenever available
        while bit_count - skip_bits > ???: #FILL IN THE BLANK - When do we have a full byte?
            result.append(buffer & ???)  # Take lowest ??? bits #FILL IN THE BLANK
            buffer >>= ???                  # Remove extracted byte #FILL IN THE BLANK
            bit_count -= 8                # Decrease bit count accordingly

    return bytes(result)

# Test Base32 decoder
test_base32 = "ph2eifo3n5utg1j8"
decoded = base32_decode_binary(test_base32)
print("Base32 Decoding Example:")
print(f"  Input:  {test_base32}")
print(f"  Output: {decoded.hex()} (hex)")
print()


SyntaxError: invalid syntax (ipython-input-364198516.py, line 43)

In [ ]:
"""
Cell 5: Substitution Cipher Decoder
The substitution cipher shifts characters by -4 positions and handles special characters.
"""

def decode_substitution_cipher(encoded_string):
    """
    Decode a domain string using SUNBURST's substitution cipher.

    Encoding rules:
    - Normal characters: shifted by -4 positions in SUBSTITUTION_ALPHABET.
    - Special characters: represented using a two-character sequence.
      The first character is in SPECIAL_CHARS, the second encodes which special char.
    """
    result = []      # List to accumulate decoded characters
    skip_next = False  # Flag indicating the next character encodes a special character

    for i, char in enumerate(encoded_string):
        if skip_next:
            # This character encodes a special character
            char_index = SUBSTITUTION_ALPHABET.find(char)  # Find position in alphabet
            if char_index != -1:
                # Map the alphabet index to the special character using modulo
                special_index = char_index % len(???) #FILL IN THE BLANK - What are we indexing into?
                result.append(SPECIAL_CHARS[special_index])
            skip_next = False  # Reset the flag
            continue  # Move to next character

        if char in SPECIAL_CHARS:
            # If current char is a special char, next char encodes the actual special char
            skip_next = ???  #FILL IN THE BLANK - What should we set the flag to?
        else:
            # Normal character: shift by ??? positions in the substitution alphabet #FILL IN THE BLANK
            char_index = SUBSTITUTION_ALPHABET.find(char)
            if char_index != -1:
                decoded_index = (char_index - ???) % len(SUBSTITUTION_ALPHABET) #FILL IN THE BLANK
                result.append(SUBSTITUTION_ALPHABET[decoded_index])

    return ''.join(result)  # Combine the list into a single decoded string

# Test substitution cipher
test_subst = "aovthro08ove0ge2h"
decoded_subst = decode_substitution_cipher(test_subst)
print("Substitution Cipher Example:")
print(f"  Input:  {test_subst}")
print(f"  Output: {decoded_subst}")
print()

SyntaxError: invalid syntax (ipython-input-2936883814.py, line 24)

In [ ]:
"""
Cell 6: GUID and Metadata Extraction
The first 16 characters contain an encrypted GUID and metadata.
The GUID identifies the victim organization, and metadata includes timestamps.
"""

TIMESTAMP_REFERENCE = datetime(2010, 1, 1, 0, 0, 0)
CAMPAIGN_START = datetime(2019, 1, 1, 0, 0, 0)
CAMPAIGN_END = datetime(2020, 12, 1, 0, 0, 0)

def decrypt_secure_string(secure_string):
    """
    Decrypt the secure string (first 15 chars) to extract GUID and XOR key.

    Returns:
        tuple: (decrypted_bytes, xor_key)
    """
    decoded_bytes = base32_decode_binary(secure_string)
    if not decoded_bytes:
        return None, None

    xor_key = decoded_bytes[???]  #FILL IN THE BLANK - Which byte contains the XOR key?
    decrypted = bytes(b ^ xor_key for b in decoded_bytes[???:]) #FILL IN THE BLANK - Which bytes to XOR?

    return decrypted, xor_key

def get_segment_number(segment_char, first_char):
    """
    Calculate the segment number for multi-part domain messages.

    Args:
        segment_char: The 16th character of the subdomain
        first_char: The first character of the subdomain

    Returns:
        Segment number (0-35)
    """
    segment_ord = ord(segment_char)
    first_ord = ord(first_char)

    if segment_ord < ???:  # Character is '0'-'9' #FILL IN THE BLANK - ASCII value of ':'
        return (segment_ord - first_ord + 96) % MAX_SEGMENTS
    else:  # Character is 'a'-'z'
        return (segment_ord - first_ord + 57) % MAX_SEGMENTS

print("GUID and Metadata Extraction:")
print("-" * 60)
# Use the first domain in the list, if available
sample_domain = domains[0] if domains else ""
if sample_domain:
    subdomain = sample_domain.split('.')[0]  # Take only the leftmost part

    if len(subdomain) > 16:
        # First ??? chars are the encrypted secure string #FILL IN THE BLANK
        secure_string = subdomain[:???] #FILL IN THE BLANK

        # 16th character encodes segment number
        segment_char = subdomain[???] #FILL IN THE BLANK - Which character position?

        # Decrypt secure string
        decrypted, xor_key = decrypt_secure_string(secure_string)

        # Check if decrypted data is valid and contains at least 8 bytes (GUID)
        if decrypted and len(decrypted) >= 8:
            guid_bytes = decrypted[:???]          # First ??? bytes are the GUID #FILL IN THE BLANK
            guid_hex = guid_bytes.hex().upper() # Represent GUID as uppercase hex
            segment_num = get_segment_number(segment_char, subdomain[0])

            print(f"Domain:      {sample_domain}")
            print(f"GUID:        {guid_hex}")
            print(f"XOR Key:     0x{xor_key:02X}" if xor_key else "N/A")
            print(f"Segment:     {segment_num}")

print()

SyntaxError: invalid syntax (ipython-input-319164738.py, line 22)

In [ ]:
"""
Cell 7: Anti-Virus Status Extraction
Some domains contain information about security products running on the victim.
"""

def extract_antivirus_status(subdomain):
    """
    Extract anti-virus product status from special SUNBURST domains.

    Returns:
        dict with extracted information or None if not an AV status domain
    """
    if len(subdomain) <= 16:
        return None

    decrypted, xor_key = decrypt_secure_string(subdomain[:15])

    if not decrypted or len(decrypted) < 11 or (xor_key & ???) != 0: #FILL IN THE BLANK - What bit mask?
        return None

    # Extract GUID with XOR operation
    guid_bytes = bytearray(8)
    for i in range(8):
        guid_bytes[i] = decrypted[i] ^ decrypted[10 - (i % ???)] #FILL IN THE BLANK - Modulo what?

    guid_string = guid_bytes.hex().upper()

    # Extract timestamp and flags
    half_hours_and_flag = ((decrypted[8] & 0x0F) << 16) | (decrypted[9] << 8) | decrypted[10]
    sz = decrypted[8] >> ???  #FILL IN THE BLANK - How many bits to shift right?

    timestamp = TIMESTAMP_REFERENCE + timedelta(minutes=??? * (half_hours_and_flag >> 1)) #FILL IN THE BLANK
    stage2 = (half_hours_and_flag & 1) == 1

    # Check if timestamp is within campaign period
    if not (CAMPAIGN_START <= timestamp <= CAMPAIGN_END):
        return None

    result = {
        'guid': guid_string,
        'timestamp': timestamp,
        'sz_type': sz,
        'stage2': stage2,
        'products': []
    }

    # Extract AV products if present
    if sz == SzType.AVProducts and len(decrypted) == ???: #FILL IN THE BLANK - Expected length?
        product_flags = (decrypted[11] << 8) | decrypted[12]

        for status in AntiVirusStatus:
            if product_flags & status:
                result['products'].append(status.name)

    return result

print("Searching for Anti-Virus Status Reports:")
print("-" * 60)
av_count = 0
for domain in domains[:100]:  # Check first 100 domains
    subdomain = domain.split('.')[0]
    av_info = extract_antivirus_status(subdomain)

    if av_info and av_info['products']:
        av_count += 1
        print(f"Found AV report in: {domain}")
        print(f"  Timestamp: {av_info['timestamp']}")
        print(f"  Products:  {', '.join(av_info['products'])}")
        if av_count >= 3:  # Show first 3 examples
            break
print(f"\nTotal AV status domains found: {av_count}")
print()

SyntaxError: invalid syntax (ipython-input-2364859060.py, line 18)

In [ ]:
"""
Cell 8: Complete Domain Decoder
Combine all decoding methods to process SUNBURST domains completely.
"""

def decode_sunburst_domain(full_domain):
    """
    Fully decode a SUNBURST domain, including GUID extraction and victim domain decoding.

    Args:
        full_domain (str): The full domain string to decode.

    Returns:
        dict: Contains decoded information:
            - type: 'domain_fragment' or 'av_status'
            - domain: original full domain
            - guid: extracted GUID (hex) for domain fragments
            - segment: segment number for multi-part domains
            - encoding: 'base32', 'substitution', or 'unknown'
            - encoded: encoded part of the domain
            - decoded: decoded victim domain string
            - xor_key: XOR key used in decryption
            - info: for 'av_status', contains antivirus info
    """
    # Split the domain into labels and take the first subdomain
    parts = full_domain.split('.')
    if not parts:
        return None

    subdomain = parts[0].strip().strip('"')

    # Subdomain must be longer than ??? chars to contain secure string + segment #FILL IN THE BLANK
    if len(subdomain) <= ???: #FILL IN THE BLANK
        return None

    # First, check if this is an AV status domain
    av_info = extract_antivirus_status(subdomain)
    if av_info:
        return {
            'type': 'av_status',
            'domain': full_domain,
            'info': av_info
        }

    # Otherwise, treat as a victim domain
    secure_string = subdomain[:15]  # Encrypted secure string
    segment_char = subdomain[15]    # 16th character encodes segment number
    encoded_part = subdomain[???:]   # Remaining characters: encoded victim domain #FILL IN THE BLANK

    # Decrypt the secure string to get GUID and XOR key
    decrypted, xor_key = decrypt_secure_string(secure_string)
    if not decrypted or len(decrypted) < 8:
        return None

    # Extract first 8 bytes as GUID
    guid_bytes = decrypted[:8]
    guid_hex = guid_bytes.hex().upper()

    # Calculate segment number for multi-part messages
    segment_num = get_segment_number(segment_char, subdomain[0])

    # Decode the victim domain part
    decoded_domain = None
    encoding_type = "unknown"

    if encoded_part.startswith("???"): #FILL IN THE BLANK - What prefix indicates Base32?
        # Base32 encoded victim domain
        encoding_type = "base32"
        try:
            decoded_bytes = base32_decode_binary(encoded_part[???:]) #FILL IN THE BLANK - Skip prefix
            decoded_domain = decoded_bytes.decode('utf-8', errors='replace')
        except Exception:
            decoded_domain = None
    elif "???" in encoded_part: #FILL IN THE BLANK - What character indicates substitution cipher?
        # Substitution cipher
        encoding_type = "substitution"
        decoded_domain = decode_substitution_cipher(encoded_part)

    # Return structured dictionary with all relevant info
    return {
        'type': 'domain_fragment',
        'domain': full_domain,
        'guid': guid_hex,
        'segment': segment_num,
        'encoding': encoding_type,
        'encoded': encoded_part,
        'decoded': decoded_domain,
        'xor_key': xor_key
    }

print("Decoding SUNBURST Domains:")
print("-" * 80)
print(f"{'Type':<15} {'GUID':<16} {'Seg':<4} {'Encoding':<12} {'Decoded'}")
print("-" * 80)

decoded_count = 0
for domain in domains[:50]:  # Process first 50 domains
    result = decode_sunburst_domain(domain)

    if result:
        if result['type'] == 'domain_fragment' and result['decoded']:
            decoded_count += 1
            print(f"{'Domain':<15} {result['guid']:<16} {result['segment']:<4} "
                  f"{result['encoding']:<12} {result['decoded'][:40]}")
        elif result['type'] == 'av_status':
            decoded_count += 1
            info = result['info']
            products = ','.join(p.split('_')[0] for p in info['products'][:3])
            print(f"{'AV Status':<15} {info['guid']:<16} {'--':<4} "
                  f"{'special':<12} {products[:40]}")

        if decoded_count >= 30:  # Show first 10 decoded
            break

print(f"\nSuccessfully decoded: {decoded_count} domains")
print()


SyntaxError: invalid syntax (ipython-input-839079380.py, line 33)

In [ ]:
"""
Cell 9: Fragment Reassembly
Victim domains longer than ~20 characters are split across multiple DNS queries.
We need to reassemble them using the GUID and segment numbers.
"""

from collections import defaultdict

def reassemble_domain_fragments(domains):
    """
    Reassemble domain fragments from multiple SUNBURST DNS queries.

    Each domain fragment contains:
        - GUID: identifies the victim
        - Segment number: position of fragment in multi-part domain
        - Decoded victim domain fragment

    Fragments with the same GUID are combined in segment order to reconstruct
    the original victim domain.

    Args:
        domains (list of str): List of full SUNBURST domains

    Returns:
        tuple:
            - reassembled (dict): GUID -> fully reassembled domain (string)
            - av_status_by_guid (dict): GUID -> AV status information (if any)
    """
    # Dictionary storing fragments by GUID; each GUID maps to a fixed-size list
    # of MAX_SEGMENTS empty strings to hold segments in order
    fragments_by_guid = defaultdict(lambda: [''] * ???) #FILL IN THE BLANK - How many slots?

    # Dictionary storing antivirus status information for each GUID
    av_status_by_guid = {}

    # Step 1: Decode each domain and sort into fragments / AV status
    for domain in domains:
        result = decode_sunburst_domain(domain)
        if not result:
            continue  # Skip invalid or undecodable domains

        if result['type'] == 'domain_fragment':
            guid = result['guid']
            segment = result['???']  #FILL IN THE BLANK - What field tells us the position?
            if result['decoded']:
                # Place decoded fragment in the correct segment slot
                fragments_by_guid[guid][???] = result['decoded'] #FILL IN THE BLANK - Where to place it?

        elif result['type'] == 'av_status':
            info = result['info']
            av_status_by_guid[info['guid']] = info

    # Step 2: Reassemble fragments for each GUID
    reassembled = {}
    for guid, fragments in fragments_by_guid.items():
        domain_parts = []

        # Combine fragments in order until a gap is found
        for fragment in fragments:
            if fragment:
                domain_parts.append(fragment)
            elif domain_parts:  # Stop at first gap after some fragments
                break

        if domain_parts:
            full_domain = ''.????(domain_parts) #FILL IN THE BLANK - How to combine strings?

            # If the domain does not end with a recognized TLD, mark as incomplete
            if not full_domain.endswith(('.com', '.org', '.net', '.gov', '.edu',
                                        '.mil', '.int', '.local')):
                full_domain += '???'  # Incomplete fragment marker #FILL IN THE BLANK

            reassembled[guid] = full_domain

    return reassembled, av_status_by_guid


# ------------------------------
# Example usage / display
# ------------------------------

print("Reassembling Domain Fragments:")
print("-" * 60)

# Reassemble domains from the provided list
reassembled_domains, av_status = reassemble_domain_fragments(domains)

print(f"Found {len(reassembled_domains)} unique GUIDs with domain fragments")
print(f"Found {len(av_status)} GUIDs with AV status reports\n")

print("Sample Reassembled Domains:")
print("-" * 60)

# Show first 10 reassembled domains
for i, (guid, domain) in enumerate(list(reassembled_domains.items())[:10]):
    incomplete = "*" if domain.endswith('*') else ""
    print(f"{guid}: {domain}{incomplete}")

    # Display AV status if available
    if guid in av_status:
        info = av_status[guid]
        products = ', '.join(p.split('_')[0] for p in info['products'])
        print(f"  └─ AV Status: {products}")
        print(f"     Stage2: {info['stage2']}, Time: {info['timestamp']}")
print()

SyntaxError: invalid syntax (ipython-input-2629702279.py, line 31)

In [ ]:
"""
Cell 10: Statistical Analysis
Analyze the decoded domains to understand the scope of the attack.
"""

def analyze_decoded_domains(reassembled_domains, av_status):
    """
    Perform statistical analysis on decoded domains.
    """
    stats = {
        'total_guids': len(reassembled_domains),
        'complete_domains': 0,
        'incomplete_domains': 0,
        'tlds': defaultdict(int),
        'av_products': defaultdict(int),
        'stage2_count': 0,
        'sectors': defaultdict(int)
    }

    # Analyze domains
    for domain in reassembled_domains.values():
        if domain.endswith('*'):
            stats['incomplete_domains'] += 1
            domain = domain[:-1]  # Remove asterisk
        else:
            stats['complete_domains'] += 1

        # Extract TLD
        if '.' in domain:
            parts = domain.split('.')
            if len(parts) > 1:
                tld = parts[-1]
                stats['tlds'][tld] += 1

        # Identify sectors
        lower = domain.lower()
        if 'gov' in lower:
            stats['sectors']['Government'] += 1
        if any(word in lower for word in ['corp', 'inc', 'llc', 'ltd']):
            stats['sectors']['Corporate'] += 1
        if any(word in lower for word in ['university', 'edu', 'academic']):
            stats['sectors']['Education'] += 1
        if any(word in lower for word in ['hospital', 'health', 'medical']):
            stats['sectors']['Healthcare'] += 1

    # Analyze AV products
    for info in av_status.values():
        if info['stage2']:
            stats['stage2_count'] += 1

        for product in info['products']:
            product_name = product.split('_')[0]
            stats['av_products'][product_name] += 1

    return stats

stats = analyze_decoded_domains(reassembled_domains, av_status)

print("SUNBURST Campaign Statistics:")
print("=" * 60)
print(f"Total unique victims (GUIDs):     {stats['total_guids']}")
print(f"Complete domain names:             {stats['complete_domains']}")
print(f"Incomplete domain fragments:       {stats['incomplete_domains']}")
print(f"Stage 2 infections:                {stats['stage2_count']}")
print()

if stats['tlds']:
    print("Top Level Domains:")
    for tld, count in sorted(stats['tlds'].items(), key=lambda x: x[1], reverse=True)[:5]:
        print(f"  .{tld:<10} {count:>5} victims")
    print()

if stats['sectors']:
    print("Targeted Sectors:")
    for sector, count in sorted(stats['sectors'].items(), key=lambda x: x[1], reverse=True):
        print(f"  {sector:<15} {count:>5} victims")
    print()

if stats['av_products']:
    print("Security Products Detected:")
    for product, count in sorted(stats['av_products'].items(), key=lambda x: x[1], reverse=True):
        print(f"  {product:<20} {count:>5} instances")
print()


SUNBURST Campaign Statistics:
Total unique victims (GUIDs):     390
Complete domain names:             127
Incomplete domain fragments:       263
Stage 2 infections:                0

Top Level Domains:
  .com           58 victims
  .local         34 victims
  .              19 victims
  .org           16 victims
  .net           10 victims

Targeted Sectors:
  Corporate          30 victims
  Healthcare          5 victims
  Government          3 victims
  Education           3 victims


